<a href="https://colab.research.google.com/github/SebsPER/TDP-Sprint3/blob/main/Const_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -q git+https://github.com/huggingface/transformers.git
!pip install transformers==4.19.4
!pip install evaluate
!pip install datasets
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 19.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.4 MB/s 
     |████████████████████████████████| 163 kB 72.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 115 kB 62.1 MB/s 
     |████████████████████████████████| 212 kB 68.4 MB/s 
     |████████████████████████████████| 441 kB 42.0 MB/s 
     |████████████████████████████████| 95 kB 5.4 MB/s 
     |████████████████████████████████| 127 kB 70.0 MB/s 
     |████████████████████████████████| 115 kB 72.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 

Create folders:

- tokenizer

- ddGPT

- ddGptModel

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, DataCollatorForLanguageModeling, Trainer, TrainingArguments, PhrasalConstraint, AutoTokenizer, AutoModelForCausalLM
#from tensorflow.keras.optimizers import Adam
from tokenizers import ByteLevelBPETokenizer
from datasets import load_dataset

In [ ]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Customize training
tokenizer.train(files="dd.txt", vocab_size=52_000, min_frequency=2, special_tokens=["<s>","<pad>","</s>","<unk>","<mask>"]) #

# Save files to disk
#tokenizer.save_model(".", "ddGPT2")
tokenizer.save_model("tokenizer")

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [ ]:
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#tokenizer = GPT2Tokenizer('ddGPT2-vocab.json', 'ddGPT2-merges.txt')
tokenizer = GPT2Tokenizer.from_pretrained('tokenizer')

#if tokenizer.pad_token is None:
#    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer.add_special_tokens({
    "eos_token": "</s>",
    "bos_token": "<s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
})

0

In [ ]:
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    bos_token=tokenizer.bos_token_id,
    eos_token=tokenizer.eos_token_id,
)
#model = GPT2LMHeadModel.from_pretrained("gpt2")
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel(config)

dataset = load_dataset("text", data_files="dd.txt")

In [ ]:
print(dataset['train'][0])

def encode(lines):
    return tokenizer(lines['text'], truncation=True, add_special_tokens=True, ) #, max_length = 100

dataset.set_transform(encode)

print(dataset['train'][0])
print("len",len(dataset['train']))
dataset = dataset['train']

t = "hello world"
en = tokenizer.encode(t)

print(en)
print(tokenizer.decode(en))
#dataset = dataset['train']

{'text': 'Vistaru, Lord of the Mountain, is attacking small towns surrounded by his kingdom in hopes of expanding his army before his assault on the capital.'}


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [58, 605, 276, 89, 16, 3176, 285, 265, 4857, 16, 299, 1365, 763, 3091, 2786, 384, 439, 1469, 297, 3268, 285, 3860, 274, 439, 1686, 714, 439, 1055, 69, 645, 334, 265, 5101, 18], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
len 1061
[263, 291, 83, 846]
hello world


In [ ]:
dataCollator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
training_args = TrainingArguments(
    output_dir="ddGPT",
    overwrite_output_dir=True,
    num_train_epochs=300,
    #per_gpu_train_batch_size=64,
    per_device_train_batch_size=20,
    save_steps=50,
    save_total_limit=2,
    prediction_loss_only=True,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=dataCollator
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("ddGptModel")

Saving model checkpoint to ddGptModel
Configuration saved in ddGptModel/config.json
Model weights saved in ddGptModel/pytorch_model.bin


# Test

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('tokenizer')
tokenizer.add_special_tokens({
    "eos_token": "</s>",
    "bos_token": "<s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
})

In [ ]:
modelT = GPT2LMHeadModel.from_pretrained("ddGptModel")

In [2]:
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("KoboldAI/GPT-Neo-1.3B-Adventure")

force_word = "scared"
force_flexible = ["scream", "screams", "screaming", "screamed"]

force_words_ids = [
    tokenizer([force_word], add_special_tokens=False).input_ids, #, return_tensors="pt",
    #tokenizer(force_flexible, padding=True, add_prefix_space=True, add_special_tokens=False).input_ids, #truncation= True, 
]

generator = pipeline('text-generation', 
                     model='KoboldAI/GPT-Neo-1.3B-Adventure',
                     num_beams=3)

In [3]:
generator("> The party", do_sample=False, min_length=50, force_words_ids=force_words_ids)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '> The party is over.scared.\n\n> You go back to your room.\nYou go back to your room.\n\nYou go back to your room.\n\nYou go back to your room.\n\nYou go back'}]

In [4]:
generator("> The party", do_sample=True, min_length=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "> The party is in a field somewhere. You're looking for a couple of your friends.'\n'Hey, no need to be rude to me.'\n'But it's rude to leave these people behind! I'm going to enjoy this,"}]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("KoboldAI/GPT-Neo-1.3B-Adventure")

modelT = AutoModelForCausalLM.from_pretrained("KoboldAI/GPT-Neo-1.3B-Adventure")

In [ ]:
force_word = "scared"
force_flexible = ["scream", "screams", "screaming", "screamed"]

force_words_ids = [
    tokenizer([force_word], add_prefix_space=True, add_special_tokens=False).input_ids, #, return_tensors="pt",
    #tokenizer(force_flexible, padding=True, add_prefix_space=True, add_special_tokens=False).input_ids, #truncation= True, 
]

starting_text = ["A mage", "A kingdom", 'The party']

input_ids = tokenizer(starting_text, return_tensors="pt").input_ids
print("INPUT:",input_ids)

outputs = modelT.generate(
    input_ids,
    #force_words_ids=force_words_ids,
    num_beams=3,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
    max_length=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(tokenizer.decode(outputs[1], skip_special_tokens=True))
print(tokenizer.decode(outputs[2], skip_special_tokens=True))

# Phrasal Constraint

In [ ]:
force_word = "scared"
force_flexible = ["scream", "screams", "screaming", "screamed"]
force_words = [" attack", " dragon"]

constraints = []
for constraint_entitiy in force_words:
    constraints.append(PhrasalConstraint(tokenizer(constraint_entitiy).input_ids))

starting_text = ["A mage", "A kingdom"]

input_ids = tokenizer(starting_text, return_tensors="pt").input_ids
print("INPUT:",input_ids)

outputs = modelT.generate(
    input_ids,
    #constraints=constraints,
    #force_words_ids=force_words_ids,
    num_beams=10,
    #num_return_sequences=1,
    #no_repeat_ngram_size=1,
    #remove_invalid_values=True,
    do_sample=True,
    max_length=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

In [ ]:
!zip -r ./tknzer.zip ./tokenizer/

  adding: tokenizer/ (stored 0%)
  adding: tokenizer/vocab.json (deflated 59%)
  adding: tokenizer/merges.txt (deflated 57%)


In [ ]:
#!zip -r ./gpt_train.zip ./ddGPT/

In [ ]:
!zip -r ./gpt_model.zip ./ddGptModel/

  adding: ddGptModel/ (stored 0%)
  adding: ddGptModel/pytorch_model.bin (deflated 10%)
  adding: ddGptModel/training_args.bin (deflated 48%)
  adding: ddGptModel/config.json (deflated 52%)


# Load

In [ ]:
!unzip tknzerV2.zip

Archive:  tknzerV2.zip
   creating: tokenizer/
  inflating: tokenizer/vocab.json    
  inflating: tokenizer/merges.txt    


In [ ]:
!unzip gpt_modelV2.zip

Archive:  gpt_modelV2.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of gpt_modelV2.zip or
        gpt_modelV2.zip.zip, and cannot find gpt_modelV2.zip.ZIP, period.
